<a href="https://colab.research.google.com/github/c-apeltauer/Eniac_and_Discounts/blob/main/Kopie_von_2_data_cleaning_with_pandas_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [1]:
import pandas as pd

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

Before we begin, let's create a copy of the `orders` and `orderlines` DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [3]:
orders_df = orders.copy()

In [4]:
orderlines_df = orderlines.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [5]:
# orders_df
orders_df.duplicated().sum()

np.int64(0)

In [6]:
# orderlines_df
orderlines_df.duplicated().sum()

np.int64(0)

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [7]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [8]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [9]:
num_missing = orders_df['total_paid'].isna().sum()
total_rows = orders_df.shape[0]
percent_missing = (100*num_missing/total_rows)
print(f"5 missing values represents {percent_missing:.5f}% of the rows in our DataFrame")

5 missing values represents 0.00220% of the rows in our DataFrame


> A quick way to find out a percentage if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [10]:
orders_df['total_paid'].isna().value_counts(normalize=True)

,proportion
total_paid,
False,0.999978
True,0.000022


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [11]:
orders_df = orders_df.dropna(axis=0)

Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames if they are redundantly stored
+ you can delete the rows or columns
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines_df`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [12]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

<ipython-input-12-87c26cc5d8b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])


### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [13]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

#### 4.1.2.&nbsp;`unit_price`

In [14]:
#orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have multiple decimal points. Let's see if there are any other numbers like this:

> `.` is a wildcard in regex, we need the `\` as an escape

In [15]:
# Count the number of decimal points in the unit_price
orderlines_df['unit_price'].str.count("\.").value_counts()

,count
unit_price,
1,257814
2,36169


In [16]:
orderlines_df['unit_price'].str.count("\d{1,2}\.\d{3,3}\.\d{2,2}").value_counts()

,count
unit_price,
0,257814
1,36169


In [17]:
orderlines_df[orderlines_df['unit_price'].str.contains("\d{1,2}\.\d{3,3}\.\d{2,2}")]

,id,id_order,product_id,product_quantity,sku,unit_price,date
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11
...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51


Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [18]:
# Count the rows with more than one `.`
mult_decimal_rows = (orderlines_df['unit_price'].str.count("\.")>1).sum()

# Find the percentage of corrupted rows
percent_corrupted = (100 * mult_decimal_rows / orderlines_df.shape[0])
print(f"{percent_corrupted:.2f}% of the rows in our DataFrame have multiple decimal points in the unit_price")

12.30% of the rows in our DataFrame have multiple decimal points in the unit_price


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [19]:
# Boolean mask to find the orders that contain a price with multiple decimal points
#multiple_decimal_mask = orderlines_df['unit_price'].str.count("\.") > 1

# Apply the boolean mask to the orderlines DataFrame. This way we can find the order_id of all the affected orders.
#corrupted_order_ids = orderlines_df.loc[multiple_decimal_mask, "id_order"]

# Keep only the rows that do not have multiple decimal points
#orderlines_df = orderlines_df.loc[~orderlines_df['id_order'].isin(corrupted_order_ids)]

In [20]:
orderlines_df.shape[0]

293983

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [21]:
orderlines_df['unit_price'] = orderlines_df['unit_price'].astype(str).apply(lambda x: x.replace('.','',1) if x.count('.')>1 else x) #removes first decimal point
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [22]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [23]:
# your code here
products_df = products.copy()
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [24]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Look for Duplicates

In [25]:
products_df.duplicated().sum()

np.int64(8746)

In [26]:
products_df = products_df.drop_duplicates()
products_df.duplicated().sum()

np.int64(0)

In [27]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Look for Missing values


In [28]:
# your code here
products_df.isna().sum()

,0
sku,0
name,0
desc,7
price,46
promo_price,0
in_stock,0
type,50


In [29]:
# Replace missing desc with empty strings
products_df.loc[:, 'desc'] = products_df.loc[:, 'desc'].fillna('')
products_df.isna().sum()

,0
sku,0
name,0
desc,0
price,46
promo_price,0
in_stock,0
type,50


In [30]:
products_df[products_df.loc[:, 'type'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
307,SAN0017,SanDisk Cruzer Edge USB 2.0 Flash Drive 16GB,Pendrive USB 2.0 Mac and PC.,8,59.895,0,NaN
530,SAN0026,SanDisk Extreme Cruzer 16GB USB 3.0 Flash Drive,USB 3.0 flash drive 16GB USB Mac and PC.,22,209.935,0,NaN
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,0,NaN
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,0,NaN
798,APP0675,Apple iPhone 5S 32GB Space Gray,New Free iPhone 5S 32GB (ME435Y / A).,559,3.989.975,0,NaN
1193,APP0823,Apple iPhone 6 16GB Silver,New iPhone 6 16GB Free (MG482QL / A).,639,639.001,0,NaN
1199,APP0829,Apple iPhone 6 Plus 16GB Silver,New iPhone 6 Plus 16G Free (MGA92QL / A).,749,7.490.021,0,NaN
1200,APP0822,Apple iPhone 6 16GB Space Gray,New iPhone 6 16GB Free (MG472QL / A).,639,639.001,0,NaN
1201,APP0825,Apple iPhone 6 64GB Space Gray,New iPhone 6 64GB Free (MG4F2QL / A).,749,7.490.021,0,NaN
1202,APP0826,Apple iPhone 6 64GB Silver,New iPhone 6 64GB Free (MG4H2QL / A).,749,7.503.331,0,NaN


In [31]:
products_df[products_df.loc[:, 'type'] == '0']

,sku,name,desc,price,promo_price,in_stock,type


In [32]:
# Replace na types with '0' (not in use otherwise)
products_df.loc[:, 'type'] = products_df.loc[:, 'type'].fillna('0')
products_df.isna().sum()

,0
sku,0
name,0
desc,0
price,46
promo_price,0
in_stock,0
type,0


### Check / Change Data types

In [33]:
# your code here
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10580 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


In [34]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [35]:
# Make in_stock boolean
products_df.loc[:, 'in_stock'] = pd.to_numeric(products_df.loc[:, 'in_stock'])
products_df.loc[:, 'in_stock'] = products_df.loc[:, 'in_stock'].astype(bool)
products_df

<ipython-input-35-f34a6c473492>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ True False False ...  True False  True]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  products_df.loc[:, 'in_stock'] = products_df.loc[:, 'in_stock'].astype(bool)


,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,True,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,False,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,False,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,False,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,True,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,True,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,False,1392


In [36]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  bool  
 6   type         10580 non-null  object
dtypes: bool(1), object(6)
memory usage: 588.9+ KB


In [37]:
# make type numerical
products_df.loc[:, 'type'] = products_df.loc[:, 'type'].str.replace(',', '.')
#products_df.info()

In [38]:
products_df['type'] = products_df.loc[:, 'type'].astype(float)
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


<ipython-input-38-c6883cd28c4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['type'] = products_df.loc[:, 'type'].astype(float)


In [39]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,True,8696.0
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,False,13855401.0
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,False,1387.0
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,False,1230.0
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,True,1364.0
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,True,12282.0
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392.0
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392.0
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,False,1392.0


In [40]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10580 non-null  object 
 1   name         10580 non-null  object 
 2   desc         10580 non-null  object 
 3   price        10534 non-null  object 
 4   promo_price  10580 non-null  object 
 5   in_stock     10580 non-null  bool   
 6   type         10580 non-null  float64
dtypes: bool(1), float64(1), object(5)
memory usage: 588.9+ KB


In [41]:
products_df['type'] = products_df.loc[:, 'type'].astype(int)
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  bool  
 6   type         10580 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 588.9+ KB


<ipython-input-41-c61ef9787496>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['type'] = products_df.loc[:, 'type'].astype(int)


In [42]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,True,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,False,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,False,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,False,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,True,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,True,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,False,1392


In [43]:
products_df.loc[products_df.loc[:, 'type'] > 1E+11]

,sku,name,desc,price,promo_price,in_stock,type
25,MOP0011,Mophie Juice Pack Air Battery Case iPhone 4 / ...,Battery Case iPhone 4 / 4S rigid polymer.,69.99,56.991,False,549000000000
58,MOP0013,Mophie Juice Pack Air Battery Case iPhone 4 / ...,Case Battery (1500mAh) iPhone 4 / 4S polymer.,69.95,539.902,False,549000000000
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,False,144000000000
369,REP0036,Full screen repair iPhone 4,Repair service including parts and labor for i...,49.99,499.899,False,144000000000
370,REP0040,Home button repair iPhone 4,Repair service including parts and labor for i...,49.99,499.899,False,144000000000
...,...,...,...,...,...,...,...
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19316,REP0407,iPad battery repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19317,REP0403,iPad LCD screen repair,Repair service including parts and labor for iPad,159.99,1.599.898,False,144000000000
19318,REP0402,iPad touch screen repair,Repair service including parts and labor for iPad,139.99,1.399.897,False,144000000000


In [44]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  bool  
 6   type         10580 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 588.9+ KB


In [45]:
products_df.isna().sum()

,0
sku,0
name,0
desc,0
price,46
promo_price,0
in_stock,0
type,0


In [46]:
#products_df = products_df.set_index('sku')
#products_df


In [47]:
#find missing prices in products
products_missing_price = products_df.loc[products_df['price'].isna(),:]
products_missing_price

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,False,8696
1900,AII0008,Aiino Case MacBook Air 11 '' Transparent,MacBook Air 11-inch casing with matte finish.,NaN,22.99,False,13835403
2039,CEL0020,Celly Ambo Luxury Leather Case + iPhone 6 Case...,Cover and housing together with magnet for iPh...,NaN,399.905,False,11865403
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,False,11865403
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,False,11865403
2044,CEL0014,Celly Silicone Hard Shell iPhone 6 Amarillo,Hard Shell Silicone iPhone 6.,NaN,59.895,False,11865403
2049,CEL0015,Celly fur-lined Powerbank battery 4000mAh Black,Leather-wrapped External Battery 4000mAh for i...,NaN,239.895,False,1515
2051,CEL0018,Celly Wallet Leather Case cover Black iPhone 6,Card case with transparent protective cover fo...,NaN,294.877,False,11865403
2052,CEL0023,Celly Ambo Luxury Leather Case + Case Gold iPh...,Cover and housing together with magnet for iPh...,NaN,329.894,False,11865403
2053,CEL0025,Celly Ambo Luxury Leather Case + Case iPhone 6...,Cover and housing together with magnet for iPh...,NaN,449.878,False,11865403


In [48]:
#find sku prices in the orderlines df
sku_prices_match =orderlines_df.loc[orderlines_df['sku'].isin(products_missing_price['sku'])]
#find the mean of the prices
reconstructed_prices = sku_prices_match.groupby('sku')['unit_price'].mean().round(2)
reconstructed_prices = reconstructed_prices.reset_index()

In [49]:
for index, row in reconstructed_prices.iterrows():
  sku = row['sku']
  price = row['unit_price']
  products_df.loc[products_df['sku'] == sku, 'price'] = str(price)


In [50]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10558 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  bool  
 6   type         10580 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 588.9+ KB


In [51]:
# Still 22 missing
# drop them
products_df = products_df.dropna(axis=0)
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10558 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10558 non-null  object
 1   name         10558 non-null  object
 2   desc         10558 non-null  object
 3   price        10558 non-null  object
 4   promo_price  10558 non-null  object
 5   in_stock     10558 non-null  bool  
 6   type         10558 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 587.7+ KB


In [52]:
products_df.loc[:, 'price'].str.contains("^\d{3,3}\.\d{3,3}$").value_counts()

,count
price,
False,10419
True,139


In [53]:
products_df.isna().sum()

,0
sku,0
name,0
desc,0
price,0
promo_price,0
in_stock,0
type,0


In [54]:
products_df.loc[products_df.loc[:, 'price'].str.contains("^\d{3,3}\.\d{3,3}$")]

,sku,name,desc,price,promo_price,in_stock,type
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,False,144000000000
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,649.891,649.891,False,5384
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,False,20642062
525,SEV0024,Service installation RAM + HDD + SSD Mac mini,installation RAM HDD + SSD + on your Mac mini ...,599.918,599.918,False,20642062
526,SEV0025,Service installation RAM + HDD + SSD iMac,installation RAM HDD + SSD + on your iMac + Da...,599.918,599.918,False,20642062
...,...,...,...,...,...,...,...
19093,ELG0037,Elgato Eve Wireless Sensor Room Interior,Sensor air quality temperature and humidity fo...,799.931,799.931,True,11905404
19162,FIB0002-A,Open - Fibaro HomeKitt Sensor Door / Window,Sensor refurbished multi-functions for doors a...,599.918,501.067,False,11859
19185,RIN0017,Chime Bell Ring,Chime bell with free Wi-Fi to amplify your not...,350.005,350.005,False,0
19298,REP0428,iPhone Speaker lower repair 7,Repair service including parts and labor for i...,599.906,599.906,False,144000000000


In [55]:
products_df.loc[products_df.loc[:, 'price'].str.contains("^\d+\.\d{3,3}\.\d{3,3}$")]

,sku,name,desc,price,promo_price,in_stock,type
665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,1.639.792,1.629.894,True,1364
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,False,0
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,False,0
827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,2.199.791,2.199.901,False,11935397
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,False,11935397
...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000


In [56]:
products_df.loc[(products_df.loc[:, 'type'] > 1E+10) & (products_df.loc[:, 'price'].str.contains("^\d+\.\d{3,3}\.\d{3,3}$"))]

,sku,name,desc,price,promo_price,in_stock,type
898,REP0156,iPhone 5 GSM antenna repair,Repair service including parts and labor for i...,69.989.909,699.899,False,144000000000
941,REP0185,Home button repair iPad mini,Repair service including parts and labor for i...,69.989.909,699.899,False,144000000000
943,REP0188,Full Screen Repair iPad Mini 2,Repair service including parts and labor for i...,2.099.895,2.099.895,False,144000000000
1057,MOP0057,Mophie Space Pack Battery Case (1700mAh) and S...,Housing with battery and 16GB external storage...,1.329.911,1.329.911,False,549000000000
1058,MOP0058,Mophie Space Pack Battery Case (1700mAh) and S...,Housing with battery and 32GB external storage...,1.599.862,1.599.862,False,549000000000
...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000


In [57]:
products_df.loc[:, 'price'].str.count("\.").value_counts(normalize=True)

,proportion
price,
1,0.659879
0,0.304414
2,0.035708


In [58]:
products_df.loc[products_df.loc[:, 'price'].str.contains("\d\.\d{3,3}\.\d{3,3}")]

,sku,name,desc,price,promo_price,in_stock,type
665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,1.639.792,1.629.894,True,1364
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,False,0
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,False,0
827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,2.199.791,2.199.901,False,11935397
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,False,11935397
...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000


In [59]:
products_df.loc[products_df.loc[:, 'price'].str.contains("\d+\.\d{3,3}")]

,sku,name,desc,price,promo_price,in_stock,type
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,False,144000000000
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,98.978,98.978,False,21485407
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,649.891,649.891,False,5384
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,False,20642062
525,SEV0024,Service installation RAM + HDD + SSD Mac mini,installation RAM HDD + SSD + on your Mac mini ...,599.918,599.918,False,20642062
...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,False,144000000000


In [60]:
import re

def deal_with_two_dots(price):
  dots = re.search("^(\d+)\.(\d{3,3})\.(\d)\d{2,2}", price)
  if dots:
    price = dots.group(1) + dots.group(2) + dots.group(3)
    return float(price)/100
  one_dot = re.search("^(\d+)\.(\d)\d{2,2}", price)
  if one_dot:
    price = one_dot.group(1) + one_dot.group(2)
    return float(price)/100
  return float(price)

products_df['price'] = products_df.agg(lambda x: deal_with_two_dots(x.loc['price']), axis=1)

<ipython-input-60-9a8ce25d8ff2>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['price'] = products_df.agg(lambda x: deal_with_two_dots(x.loc['price']), axis=1)


In [61]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,True,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,589.996,False,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,569.898,False,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,229.997,False,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,True,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,True,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,True,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,False,1392


In [62]:
products_df.loc[:, 'promo_price'].str.contains('^\d+($|\.\d{2,2}$)').value_counts(normalize=True)

<ipython-input-62-cf2fbc711dbc>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  products_df.loc[:, 'promo_price'].str.contains('^\d+($|\.\d{2,2}$)').value_counts(normalize=True)


,proportion
promo_price,
False,0.923186
True,0.076814


In [63]:
products_df.loc[products_df.loc[:, 'price'] > 100]

,sku,name,desc,price,promo_price,in_stock,type
50,APP0367,Apple Mini DisplayPort to DVI Adapter Mac dual...,Adapter Mini Display Port to DVI dual channel ...,119.00,1.119.976,False,1325
51,APP0344,"Apple Thunderbolt Display 27 ""Monitor Mac",Monitor Display 27-inch Apple Thunderbolt (MC9...,1149.00,10.449.923,False,1296
66,MAK0008,Maclocks theft case iPad 2 3 and 4 black with ...,Case antitheft iPad 2 3 and 4 polycarbonate ro...,120.00,1.199.957,False,12635403
67,MAK0007,Maclocks theft case iPad 2 3 and 4 transparent...,Case antitheft iPad 2 3 and 4 polycarbonate ro...,120.00,1.079.961,False,12635403
73,SYN0048,Synology Pack 4 IP camera licenses,4 Pack licenses Mac and PC camera.,181.99,181.984,False,1404
...,...,...,...,...,...,...,...
19316,REP0407,iPad battery repair,Repair service including parts and labor for iPad,699.90,69.99,False,144000000000
19317,REP0403,iPad LCD screen repair,Repair service including parts and labor for iPad,159.99,1.599.898,False,144000000000
19318,REP0402,iPad touch screen repair,Repair service including parts and labor for iPad,139.99,1.399.897,False,144000000000
19319,KNO0032,"Knomo MacBook Pro Beauchamp Backpack 14 ""Black",Backpack thin nylon mesh internal compartment ...,179.00,1.699.905,True,1392


In [64]:
products_df['promo_price'] = products_df.agg(lambda x: deal_with_two_dots(x.loc['promo_price']), axis=1)

<ipython-input-64-5cec440321fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['promo_price'] = products_df.agg(lambda x: deal_with_two_dots(x.loc['promo_price']), axis=1)


In [65]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


In [66]:
products_df['price'] = products_df.loc[:, 'price'].astype(float)
products_df['promo_price'] = products_df.loc[:, 'promo_price'].astype(float)
products_df['in_stock'] = products_df.loc[:, 'in_stock'].astype(bool)
products_df['type'] = products_df.loc[:, 'type'].astype(int)

<ipython-input-66-9145fc61fa5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['price'] = products_df.loc[:, 'price'].astype(float)
<ipython-input-66-9145fc61fa5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['promo_price'] = products_df.loc[:, 'promo_price'].astype(float)
<ipython-input-66-9145fc61fa5a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [67]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10558 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10558 non-null  object 
 1   name         10558 non-null  object 
 2   desc         10558 non-null  object 
 3   price        10558 non-null  float64
 4   promo_price  10558 non-null  float64
 5   in_stock     10558 non-null  bool   
 6   type         10558 non-null  int64  
dtypes: bool(1), float64(2), int64(1), object(3)
memory usage: 587.7+ KB


In [68]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,49.98,True,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,58.99,False,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,56.98,False,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,22.99,False,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,True,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,26.99,True,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,64.99,True,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,64.99,True,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,64.99,False,1392


In [69]:
products_df.loc[products_df.loc[:, 'desc'].str.contains("iPad")]

,sku,name,desc,price,promo_price,in_stock,type
11,SEN0021,Sennheiser CX 300-II Precision headphones iPho...,Headphones iPhone iPad iPad 2 iPad 3 and iPod.,49.99,44.98,False,5384
22,JMO0014,Just Mobile Upstand iPad support,Support Retina iPad and iPad mini aluminum.,49.95,39.99,False,1216
32,BEL0070,Music Receiver Belkin iPhone music receiver,Music Receiver iPhone iPad and iPod Touch for ...,49.99,29.98,False,11905404
36,APP0308,AV Cable Adapter Apple iPhone iPad and iPod white,IPhone iPad iPod adapter and AV cable.,45.00,35.99,False,1230
39,JMO0026,Just Mobile Lazy Couch Support Mac and iPad,Mac and iPad small lift stand.,19.95,19.98,False,8696
...,...,...,...,...,...,...,...
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,699.90,69.99,False,144000000000
19316,REP0407,iPad battery repair,Repair service including parts and labor for iPad,699.90,69.99,False,144000000000
19317,REP0403,iPad LCD screen repair,Repair service including parts and labor for iPad,159.99,159.98,False,144000000000
19318,REP0402,iPad touch screen repair,Repair service including parts and labor for iPad,139.99,139.98,False,144000000000


That looks like a plausible interpretation of promo_price; but there's no guarantee that this interpretation is correct. Furthermore promo_price without any information when it is in effect, is of very little value.
Therefore promo_price should be completely dropped.